## 并发编程
### 线程和进程
操作系统可以同时执行多个任务，每一个任务就是一个进程。进程可以可以同时执行多个任务，每一个任务就是一个线程。一个程序运行后至少有一个进程，在一个进程中可以包含多个线程，但至少要包含一个主线程。
### 并发和并行
并行指在同一时刻有多条指令在多个处理器上同时执行。并发指在同一时刻只能有一条指令执行，但多个指令被快速轮换执行，使得在宏观上具有多个进程同时执行的效果。多线程就是让多个函数能并发执行，让普通用户感觉到多个函数似乎在同时执行。
### 线程的创建和启动

In [1]:
import threading

# 定义一个普通的action方法，该方法准备作为线程执行体
def action(max):
    for i in range(max):
        # 调用threading模块的current_thread()函数获取当前线程
        # 调用线程对象的getName()方法获取当前线程的名字
        print(threading.current_thread().getName() + " " + str(i))


# 下面是主程序（也就是主线程的线程执行体）
for i in range(10):
    print(threading.current_thread().getName() + " " + str(i))
    if i == 5:
        # 创建并启动第一个线程
        t1 = threading.Thread(target=action, args=(3,))
        t1.start()
        # 创建并启动第二个线程
        t2 = threading.Thread(target=action, args=(3,))
        t2.start()
print('main thread is done!')

MainThread 0
MainThread 1
MainThread 2
MainThread 3
MainThread 4
MainThread 5
Thread-4 0
Thread-4 1
Thread-4 2
Thread-5 0
Thread-5 1
Thread-5 2
MainThread 6
MainThread 7
MainThread 8
MainThread 9
main thread is done!


## 线程的生命周期
### 新建和就绪状态  
当程序创建了一个Thread对象或者Thread子类的对象后，该线程就处于新建状态，此时的线程对象没有表现出任何线程的动态特征，程序也不会执行线程执行体。当线程对象调用start()方法后，该线程就处于就绪状态，Python解释器会为其创建方法调用栈和程序计数器，处于就绪状态中的线程并没有开始执行，只是表示线程可以运行了，何时运行取决于Python解释器中线程调度器的调度。注意: 
  
1、启动线程使用start方法，而不是run方法。如果直接调用线程对象的run方法，系统会把线程对象当成一个普通对象，且在该方法返回之前其他线程无法并发执行。    
2、只能对处于新建状态的线程调用start方法，如果程序对同一个线程重复调用start方法，将引发RuntimeError异常。

### 运行和阻塞状态
如果处于就绪状态的线程获得了CPU，开始执行run()方法的线程执行体，则该线程处于运行状态。当发生以下情况时，线程会进入阻塞状态:  

1、线程调用sleep()方法主动放弃其所占用的处理器资源。  
2、线程调用了一个阻塞式I/O方法，在该方法返回之前，该线程被阻塞。  
3、线程视图获得一个锁对象，但该锁对象正被其他线程所持有。  
4、线程正在等待某个通知(Notify)。  

当前正在执行的线程被阻塞之后，其他线程就可以获得执行的机会，被阻塞的线程在合适的时间会重新进入就绪状态，等待线程调度器再次调度它:

1、调用sleep()方法的线程经过了指定的时间。  
2、线程调用的阻塞式I/O方法已经返回。  
3、线程成功获得了试图获得的锁对象。  
4、线程正在等待某个通知，其他线程发出了一个通知。

### 线程死亡
线程会以以下三种方式结束，结束后就处于死亡状态:

1、run()方法或代表线程执行体的target函数执行完成，线程正常结束。  
2、线程抛出一个未捕获的Exception或Error。  

测试某个线程是否死亡，可以调用线程对象的is_alive()方法，当线程处于就绪、运行和阻塞状态时，返回True；当线程处于新建、死亡两种状态时，返回False。不要对处于死亡状态的线程调用start()方法，会引发RuntimeError异常。

### join线程
当在某个程序执行流中调用其他线程的join()方法时，调用线程将被阻塞，直到被join()方法加入的join线程执行完成。

In [2]:
# 启动子线程
threading.Thread(target=action, args=(3,), name="new thread").start()
for i in range(10):
    if i == 5:
        jt = threading.Thread(target=action, args=(3,), name="join's thread")
        jt.start()
        # 主线程调用了jt线程的join()方法
        # 主线程必须等待jt执行结束才会向下执行
        jt.join()
    print(threading.current_thread().name + ' ' + str(i))

new thread 0
new thread 1
new thread 2
MainThread 0
MainThread 1
MainThread 2
MainThread 3
MainThread 4
join's thread 0
join's thread 1
join's thread 2
MainThread 5
MainThread 6
MainThread 7
MainThread 8
MainThread 9


### 线程睡眠

In [3]:
import time

for i in range(10):
    print('current time: %s' % time.ctime)
    time.sleep(2)

current time: <built-in function ctime>
current time: <built-in function ctime>
current time: <built-in function ctime>
current time: <built-in function ctime>
current time: <built-in function ctime>
current time: <built-in function ctime>
current time: <built-in function ctime>
current time: <built-in function ctime>
current time: <built-in function ctime>
current time: <built-in function ctime>


### 线程同步 --> 线程安全问题: 以银行取款为例

In [11]:
class Account:
    def __init__(self, account_no, balance):
        self.account_no = account_no
        self.balance = balance

# 定义一个函数模拟取钱操作
def draw(account, draw_amount):
    if account.balance >= draw_amount:
        print(threading.current_thread().name + "取钱成功！取出：" + str(draw_amount), end='\t')
        time.sleep(0.001)
        # 修改余额
        account.balance -= draw_amount
        print("余额为：" + str(account.balance))
    else:
        print(threading.current_thread().name + "取钱失败！余额不足！")

In [10]:
acct = Account('1234567', 1000)
# 使用两个线程模拟从同一个账户取钱
threading.Thread(name='甲', target=draw, args=(acct, 800)).start()
threading.Thread(name='乙', target=draw, args=(acct, 800)).start()

甲取钱成功！取出：800	余额为：200
乙取钱失败！余额不足！


In [12]:
acct1 = Account('1234567', 1000)
# 使用两个线程模拟从同一个账户取钱，并取消time.sleep(0.001)的注释
threading.Thread(name='甲', target=draw, args=(acct1, 800)).start()
threading.Thread(name='乙', target=draw, args=(acct1, 800)).start()

甲取钱成功！取出：800	乙取钱成功！取出：800	余额为：200
余额为：-600


### 同步锁(Lock)
之所以出现上述错误，是因为run()方法的方法体不具有线程安全性：程序中有两个并发线程在修改Account对象，而且系统恰好在time.sleep(0.001)处执行线程切换，切换到另一个修改Account对象的线程，所以就出现了问题。为了解决这个问题，threading模块引入了锁。threading模块提供了Lock和RLock两个类，在实现线程安全的控制中，比较常用的是RLock。

In [13]:
class Account:
    def __init__(self, account_no, balance):
        self.account_no = account_no
        self._balance = balance
        # 定义一个RLock对象
        self.lock = threading.RLock()

    # 因为账户余额不允许随意修改，所以只为self._balance提供getter方法
    def getBalance(self):
        return self._balance

    # 提供一个线程安全的draw()方法来完成取钱操作
    def draw(self, draw_amount):
        self.lock.acquire()    # 加锁
        try:
            if self._balance >= draw_amount:
                print(threading.current_thread().name + "取钱成功！取出：" + str(draw_amount), end='\t')
                time.sleep(0.001)
                self._balance -= draw_amount
                print("余额为：" + str(self._balance))
            else:
                print(threading.current_thread().name + "取钱失败！余额不足！")
        # 使用finally块来保证释放锁
        finally:
            self.lock.release()

当一个线程在draw()方法中对RLock对象加锁之后，其他线程由于无法获取对RLock对象的锁定，因此它们不能同时执行draw()方法对self._balance进行修改。这意味着：并发线程在任意时刻只有一个线程可以进入修改共享资源的代码区(也称为临界区)，所以在同一时刻最多只有一个线程处于临界区内，从而保证了线程安全。为了保证Lock对象能真正锁定它所管理的Account对象，程序会被编写成每个Account对象有一个对应的Lock:

In [14]:
def draw(account, draw_amount):
    account.draw(draw_amount)


acct = Account('1234567', 1000)
threading.Thread(name='甲', target=draw, args=(acct, 800)).start()
threading.Thread(name='乙', target=draw, args=(acct, 800)).start()

甲取钱成功！取出：800	余额为：200
乙取钱失败！余额不足！
